In this script, I firstly use simple decision tree to find important features from feature texts, and then I  use regular expression to explore those potentially important features and generate new features for future use.

This is my first post. I am new to the area, so please kindly provide me with pointers and advice. Thanks a lot.

Some codes are quoted from @sudalairajkumar post.

In [ ]:
import pandas as pd
import numpy as np
import re
import itertools as it

In [ ]:
train=pd.read_json('../input/train.json')
train['listing_id']=train['listing_id'].apply(str)

### Gather and observe all original features

In [ ]:
feature_total=[]
train['features'].apply(lambda x: feature_total.append(x))
feature_total=list(it.chain.from_iterable(feature_total))
len(feature_total)

In [ ]:
uniq_feature_total=set(feature_total)
len(uniq_feature_total)

In [ ]:
list(uniq_feature_total)[:10]

#### Findings:
1. For many listings, the features are linked with \*, which should be separated.
2. The uppercase letters should be tranfered to lowercase, which will be taken care of later. 

In [ ]:
def feature_star_sep(feature_list):
    '''
    Seperate feature text with * or • as separator
    '''
    new_list=[]
    for feature in feature_list:
        if feature[:2]=='**':
            new=feature[3:-3]
            new_list+new.split(" * ")
        elif feature[:1]=='•':
            new=feature[2:]
            new_list+new.split(" • ")            
        else:
            new_list.append(feature)
            
    return new_list

In [ ]:
train['features']=train['features'].apply(feature_star_sep)

### Adopt DecisionTree on feature text

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
## Code copied from @sudalairajkumar 
vec=CountVectorizer(stop_words='english', max_features=200)
train['features_new'] = train["features"].apply(lambda y: " ".join(["_".join(x.split(" ")).lower() for x in y]))
tr_sparse = vec.fit_transform(train["features_new"])
feature_names=vec.get_feature_names()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import log_loss

In [ ]:
target_num_map = {'high':0, 'medium':1, 'low':2}
features=tr_sparse.toarray()
labels=train['interest_level'].apply(lambda x: target_num_map[x]).as_matrix()

In [ ]:
clf=DecisionTreeClassifier(max_depth=5)

In [ ]:
cv=StratifiedShuffleSplit(n_splits=3, test_size=0.3)

for train_idx, test_idx in cv.split(features, labels): 
    features_train,labels_train = features[train_idx],labels[train_idx]
    features_test,labels_test = features[test_idx],labels[test_idx]
    clf.fit(features_train,labels_train)
    print("log loss:",(-1)*round(log_loss(labels_test,clf.predict_proba(features_test)),3))
    
    ## Print out features with high importance
    print('high importance features:')
    for idx in np.where(clf.feature_importances_>0.05)[0]:
        print("  ",feature_names[idx],round(clf.feature_importances_[idx],3))
        

### Check important features

Some important features:

- hardwood floors
- doorman
- reduced fee
- no fee

Other candidates:

- war
- laundry
- fitness/gym

In [ ]:
feature_total=[]
train['features'].apply(lambda x: feature_total.append(x))
feature_total=list(it.chain.from_iterable(feature_total))
uniq_feature_total=list(set(feature_total))

In [ ]:
def re_search(key):
    '''
    Present all features with specific re pattern
    '''
    result=[]
    my_reg=r""+key
    for item in uniq_feature_total:
        if re.compile(my_reg ,re.IGNORECASE).search(item)!=None:
            result.append(item)
    return result

In [ ]:
# Check all text including 'hardwood'
re_search('hardwood')

In [ ]:
# Check all text including 'doorman'
re_search('doorman')

In [ ]:
# Check all text including 'fee'
re_search('fee')

In [ ]:
# Extract no fee
re_search('no\s*\w*\s*fee')

In [ ]:
# Extract low fee
re_search('reduce|low\sfee')

In [ ]:
# Check all text including 'laundry'
re_search('laundry')

In [ ]:
# Extract war and exclude other keyword with 'war' such as warmth and wardrobe
re_search('war\Z|war\s')

In [ ]:
# Check all text including 'fitness' or 'gym'
re_search('fitness|gym')

### Build new features

In [ ]:
def add_feature(row):
    if re.search(r'hardwood',row['features_new'],re.IGNORECASE)!=None:
        row['hardwood']=1
    else:
        row['hardwood']=0
        
    if re.search(r'doorman',row['features_new'],re.IGNORECASE)!=None:
        row['doorman']=1
    else:
        row['doorman']=0
    
    if re.search(r'no\w*fee',row['features_new'],re.IGNORECASE)!=None:
        row['no_fee']=1
    else:
        row['no_fee']=0
    
    if re.search(r'reduce|low\wfee',row['features_new'],re.IGNORECASE)!=None:
        row['reduce_fee']=1
    else:
        row['reduce_fee']=0

    if re.search(r'laundry',row['features_new'],re.IGNORECASE)!=None:
        row['laundry']=1
    else:
        row['laundry']=0

    if re.search(r'war\Z|war\s|war_',row['features_new'],re.IGNORECASE)!=None:
        row['war']=1
    else:
        row['war']=0

    if re.search(r'fitness|gym',row['features_new'],re.IGNORECASE)!=None:
        row['gym']=1
    else:
        row['gym']=0
        
    return row

In [ ]:
train=train.apply(add_feature,axis=1)

In [ ]:
train[['hardwood','doorman','no_fee','reduce_fee','laundry','war','gym']].apply(sum)